In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from itertools import product
import requests
import time
from datetime import datetime
import logging

# Suppress yfinance logging
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

def DataFetcher(symbol, interval='1d'):
    # Check if the symbol is an integer
    if symbol.isdigit():
        return DataFetcherFallback(symbol, interval)

    suffixes = ['.NS', '.BO']
    for suffix in suffixes:
        try:
            data = yf.download(symbol + suffix, interval=interval, progress=False)
            data.drop(columns='Adj Close', inplace=True)
            data.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True)
            data['Time'] = data.index
            return data
        except Exception as e:
            print(f"Failed to fetch data for {symbol+suffix}: {e}")
            continue

    # If all attempts with yfinance fail, use the Moneycontrol API
    return pd.DataFrame()

def DataFetcherFallback(symbol, interval='1D'):
    url = "https://priceapi.moneycontrol.com/techCharts/indianMarket/stock/history?symbol={}&resolution={}&from=1612029544&to={}&countback=100000000&currencyCode=INR".format(
        symbol, interval, int(time.mktime(datetime.now().timetuple()))
    )
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        data.pop('s')
        df = pd.DataFrame(data)
        cols = {'t': 'Time', 'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}
        df.rename(columns=cols, inplace=True)
        df['Time'] = pd.to_datetime(pd.to_datetime(df['Time'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')) + pd.Timedelta(minutes=330)
        df = df.sort_values(by='Time', ascending=True)
        return df
    except Exception as e:
        print(f"Failed to fetch data from Moneycontrol API for {symbol}: {e}")
        return pd.DataFrame()


def calculate_super_trend(df, multiplier, period):
    df = df.reset_index(drop=True)  # Reset the index to ensure it's integer-based
    df['tr'] = np.maximum(df['high'] - df['low'],
                          np.abs(df['high'] - df['close'].shift(1)),
                          np.abs(df['low'] - df['close'].shift(1)))
    df['atr'] = df['tr'].rolling(window=period).mean()
    df['volume_factor'] = df['volume'] / df['volume'].rolling(window=period).mean()

    upperband = np.zeros(len(df))
    lowerband = np.zeros(len(df))
    super_trend = np.zeros(len(df))
    in_uptrend = [True] * len(df)

    for i in range(1, len(df)):
        current_close = df['close'].iloc[i]
        previous_close = df['close'].iloc[i - 1]
        previous_super_trend = super_trend[i - 1]

        if current_close > previous_super_trend:
            in_uptrend[i] = True
        else:
            in_uptrend[i] = False

        if in_uptrend[i]:
            lowerband[i] = ((df['high'].iloc[i] + df['low'].iloc[i]) / 2) - (multiplier * df['atr'].iloc[i] * df['volume_factor'].iloc[i])
            super_trend[i] = max(lowerband[i], previous_super_trend) if in_uptrend[i - 1] else lowerband[i]
        else:
            upperband[i] = ((df['high'].iloc[i] + df['low'].iloc[i]) / 2) + (multiplier * df['atr'].iloc[i] * df['volume_factor'].iloc[i])
            super_trend[i] = min(upperband[i], previous_super_trend) if not in_uptrend[i - 1] else upperband[i]

    df['super_trend'] = super_trend
    df['upperband'] = upperband
    df['lowerband'] = lowerband
    df['signal'] = np.where(df['close'] > df['super_trend'], 1, -1)
    df['Position'] = df['signal'].diff()
    df['Buy/Sell'] = df['Position'].apply(lambda x: 'Buy' if x == 2 else 'Sell' if x == -2 else 'Hold')
    return df




def calculate_super_trend_performance(df):
    if 'signal' not in df.columns:
      raise ValueError("Signal column not created. Check calculate_super_trend function.")

    df['returns'] = np.log(df['close'] / df['close'].shift(1))
    df['strategy'] = df['signal'].shift(1) * df['returns']
    df['creturns'] = df['returns'].cumsum().apply(np.exp)
    df['cstrategy'] = df['strategy'].cumsum().apply(np.exp)
    perf = df['cstrategy'].iloc[-1]
    buy_and_hold = df['creturns'].iloc[-1]
    outperf = perf - buy_and_hold
    return round(perf, 6), round(buy_and_hold, 6), round(outperf, 6)

def optimize_super_trend_parameters(df, multipliers, periods):
    best_performance = -np.inf
    best_params = None
    for multiplier, period in product(multipliers, periods):
        temp_df = df.copy()
        temp_df=calculate_super_trend(temp_df, multiplier, period)
        performance = calculate_super_trend_performance(temp_df)[0]
        if performance > best_performance:
            best_performance = performance
            best_params = (multiplier, period)
    return best_params

def calculate_profit_percentage(df):
    initial_principal = 100000  # Start with $100,000
    cash = initial_principal
    shares = 0

    for index, row in df.iterrows():
        if row['Buy/Sell'] == 'Buy' and cash > 0:
            shares_bought = cash // row['close']
            shares += shares_bought
            cash -= shares_bought * row['close']
        elif row['Buy/Sell'] == 'Sell' and shares > 0:
            cash += shares * row['close']
            shares = 0

    # If there are any shares left, sell them at the last recorded price
    if shares > 0:
        cash += shares * df['close'].iloc[-1]
        shares = 0

    final_principal = cash
    profit = final_principal - initial_principal
    profit_percentage = (profit / initial_principal) * 100
    return profit_percentage

def get_market_cap(symbol):
    def fetch_market_cap(ticker_symbol):
        try:
            ticker = yf.Ticker(ticker_symbol)
            info = ticker.info
            market_cap = info.get('marketCap')
            return market_cap
        except Exception:
            return None

    market_cap = fetch_market_cap(symbol + '.NS')

    if market_cap is None:
        market_cap = fetch_market_cap(symbol + '.BO')

    if market_cap is None:
        return -1

    # Convert market cap to crores (1 crore = 10 million)
    market_cap_in_crores = market_cap / 10**7
    return market_cap_in_crores

In [2]:
stocks_df=pd.read_csv('filtered_data.csv')

In [3]:
from tqdm import tqdm
all_stocks_data = []  # Initialize an empty list to store individual stock data
total_stocks = stocks_df.shape[0] #
failed_stocks=[]
success_stocks=[]

multipliers = np.arange(1.5, 3.5, 0.5)
periods = np.arange(7, 22, 1)

for index, row in tqdm(stocks_df.iterrows(), desc="Fetching data", unit="stock" , total=total_stocks):

    if pd.notna(row['Security Code']):
        df=DataFetcher(str(int(row['Security Code'])))
        if not df.empty:
            best_params = optimize_super_trend_parameters(df.copy(), multipliers, periods)
            temp_df = calculate_super_trend(df, best_params[0], best_params[1])
            profit_percentage = calculate_profit_percentage(temp_df)
            all_stocks_data.append({'symbol': row['symbol'] + ':'  +str(row['Security Code']), 'multiplier': best_params[0], 'period': best_params[1] , 'percentage': profit_percentage, 'Market Cap cr' : get_market_cap(row['symbol'])})
        
    else:
        df=DataFetcher(row['symbol'])
        if not df.empty:
            best_params = optimize_super_trend_parameters(df.copy(), multipliers, periods)
            temp_df = calculate_super_trend(df, best_params[0], best_params[1])
            profit_percentage = calculate_profit_percentage(temp_df)
            all_stocks_data.append({'symbol': row['symbol'], 'multiplier': best_params[0], 'period': best_params[1] , 'percentage': profit_percentage, 'Market Cap cr' : get_market_cap(row['symbol'])})


all_stocks_df = pd.DataFrame(all_stocks_data)  # Concatenate all stock dataframes


Fetching data:   0%|          | 0/4154 [00:00<?, ?stock/s]

Fetching data:   4%|▍         | 178/4154 [25:44<12:59:58, 11.77s/stock]C:\Users\Bhuvanesh\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Bhuvanesh\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Bhuvanesh\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Bhuvanesh\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Bhuvanesh\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encoun

TypeError: 'NoneType' object is not subscriptable

In [ ]:
csv_file_path = 'Paramns_stocks_1.csv'
all_stocks_df.to_csv(csv_file_path, index=False)